In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import batman
import random
from astropy.io import ascii
from scipy import constants

In [2]:
#Define constants
M_sun = 1.989e+30
R_sun = 695700000
R_Earth = 6378100
day = 86400
AU = 1.496e+11

In [3]:
def planetmaker(rates, radius_low, radius_up, period_low, period_up):
    '''Generates parameters for a given number of planets in a planetary system from occurence rates'''
    rowdex = np.arange(len(rates))
    cosi = np.random.random_sample()
    anglew = np.random.uniform(low=-180., high=180.)
    e = np.random.beta(a=1.03, b=13.6)
    rows = np.random.choice(rowdex, p=rates)
    radp = np.random.uniform(low=radius_low[rows], high=radius_up[rows])
    per = np.random.uniform(low=period_low[rows], high=period_up[rows])
    return radp, per, cosi, anglew, e

In [4]:
table = ascii.read('norm_rates_FGK.csv', format='csv', header_start=0, data_start=1)
occ = np.array(table['f'], dtype=float)
radbinlower = np.array(table['rmin'], dtype=float)
radbinupper = np.array(table['rmax'], dtype=float)
pbinlower = np.array(table['pmin'], dtype=float)
pbinupper = np.array(table['pmax'], dtype=float)

In [5]:
#Generate a random temperature. In the final pipeline Temperature will be chosen from TICv8
T_eff = np.random.uniform(3700.0, 7500.0)

In [6]:
#Assign spectral type based on temperature
if 7500<= T_eff <10000:
    spectral_type = 'A'
if 6000<= T_eff <7500:
    spectral_type = 'F'
if 5200<= T_eff <6000:
    spectral_type = 'G'
if 3700<= T_eff < 5200:
    spectral_type = 'K'
if 2400<= T_eff < 3700:
    spectral_type = 'M'

In [7]:
#Choose mass and radius dependent on spectral type. In the final pipeline these will come from TICv8
if spectral_type == 'A':
    M_star = np.random.uniform(2.1, 1.4)
    R_star = np.random.uniform(1.4, 1.8)
if spectral_type == 'F':
    M_star = np.random.uniform(1.04, 1.4)
    R_star = np.random.uniform(1.15, 1.4)
if spectral_type == 'G':
    M_star = np.random.uniform(0.8, 1.04)
    R_star = np.random.uniform(0.96, 1.15)
if spectral_type == 'K':
    M_star = np.random.uniform(0.45, 0.8)
    R_star = np.random.uniform(0.7, 0.96)
if spectral_type == 'M':
    M_star = np.random.uniform(0.08, 0.45)
    R_star = np.random.uniform(0.1, 0.8)

In [9]:
radius, period, cosi, peri, ecc = planetmaker(occ, radbinlower, radbinupper, pbinlower, pbinupper)
a = np.cbrt((constants.G*(M_star*M_sun)*np.square(period*day))/(4*np.square(np.pi)))
b = ((a*np.square(cosi))/(R_star*R_sun))*((1-np.square(ecc))/(1+ecc*np.sin(peri)))
k = (radius*R_Earth)/(R_star*R_sun)
Transit = (0<= b) & (b< 1)
Grazing = (b >= 1) & (b < (1+k))
inc = np.degrees(np.arccos(cosi))

In [10]:
if Transit == True:
    #Generate range of Time values to test for in/out transit times in TESS BJD, assuming a mission duration of 10 years and a cadence of 20s
    T_start = 0.0
    T_end = 3650.0
    cadence = 20.0
    T_range = np.arange(T_start, T_end, step=cadence/day, dtype=float)
    #Calculate Transit duration
    T_dur = (period*day/np.pi)*np.arcsin((R_star*R_sun/a)*np.sqrt(1+k-np.square(b))/np.sqrt(1-np.square(cosi)))
    T_0 = np.random.uniform(low=T_start, high=T_start+period) #First possible to observe transit ingress time
    T_1s = np.arange(T_0, T_end, period) #All transit ingress times over duration of TESS mission
    T_4s = T_1s + T_dur/day #All transit egress times over duration of TESS mission

In [13]:
print(spectral_type, 'type star')
print('Planet')
print('Radius = ',radius, 'Earth Radii')
print('Period =', period, 'days')
print('Inclination angle =', inc, 'deg')
print('Eccentricity =', ecc)
print('Semi major axis =',a/AU, 'AU')
print('Impact parameter, b =', b)
print('Transit?', Transit)
print('Grazing?', Grazing)
if Transit == True:
    print('Transit duration =', T_dur[i]/3600, 'hours')
    #print('Transits (ingress) occur at TBJD =', T_1s)

F type star
Planet
Radius =  0.6061099887634434 Earth Radii
Period = 365.00963561194783 days
Inclination angle = 69.43987241912315 deg
Eccentricity = 0.041609665538543505
Semi major axis = 1.0346468320947166 AU
Impact parameter, b = 23.01803428275487
Transit? False
Grazing? False
